# Section 3.1: Setup and Introduction

| **Aspect** | **Details** |
|-------------|-------------|
| **Goal** | Configure environment and prepare for building production-ready SDLC automation templates |
| **Time** | ~15 minutes |
| **Prerequisites** | Module 2 completion, Python 3.8+, IDE with notebook support, API access (GitHub Copilot, CircuIT, or OpenAI) |
| **Next Steps** | [Section 3.2: Code Review Automation](./3.2-code-review-automation.ipynb) |

---

**Module 3 Overview**

Module 3 is structured as three core sections plus one optional advanced section (~95-135 minutes total). Each section builds on the last, so complete them in order.

**Core Sections:**
1. [Section 3.1 - Setup and Introduction](./3.1-setup-and-introduction.ipynb) (this notebook) - 15 minutes  
   Configure dependencies, provider access, and prepare for SDLC automation.
2. [Section 3.2 - Code Review Automation](./3.2-code-review-automation.ipynb) - 40 minutes  
   Build multi-dimensional review templates with severity classification.
3. [Section 3.3 - Test Generation Automation](./3.3-test-generation-automation.ipynb) - 40 minutes  
   Generate comprehensive test specifications with ambiguity detection.

**Advanced Section:**
4. [Section 3.4 - Evaluation Templates](./3.4-evaluation-templates.ipynb) - 40 minutes  
   Build weighted rubrics and automated quality gates for production use.

**Recommended Path:** Complete Sections 3.1–3.3 first, then use Section 3.4 when you need production-grade evaluation and quality gates.

## What You'll Build: Prompt Templates

You're going to build **reusable prompt templates**. Think of them like functions—write the prompt structure once, reuse it everywhere.

Think of templates like functions: you define the prompt structure once, then call it with different inputs (code, requirements, context) to get consistent, high-quality outputs.

**Why templates?**

Modern AI coding assistants often use SKILLs (for dynamic context retrieval) or agent-based architectures (for autonomous task planning). We're using templates because:

- Standard API calls, any LLM provider, no special infrastructure
- Many teams (especially CircuIT users) don't have access to SKILLs/agent platforms
- The patterns you learn here translate directly to advanced architectures

Start with templates to get automation working today. Upgrade to SKILLs or agents when you need dynamic context management or autonomous planning.

---

## Ready to Start?

**Important:** This module builds on Module 2 tactics. If you haven't completed Module 2, start there first. Even if you did, run the setup cells below to confirm everything is configured.

## Setup: Environment Configuration

### Step 1: Install Required Dependencies

Install the required packages. Run the cell below.


In [ ]:
# Install required packages for Module 3
import subprocess
import sys

try:
    subprocess.run(
        [sys.executable, "-m", "pip", "install", "-r", "requirements.txt"],
        check=True
    )
    print("\nSUCCESS: All dependencies installed.")
except subprocess.CalledProcessError:
    print("\nInstallation failed.")
    print("\nTroubleshooting:")
    print("   1. Ensure pip is installed: python -m ensurepip --upgrade")
    print("   2. Try manual install: pip install openai anthropic python-dotenv requests")
except Exception as e:
    print(f"\nError: {e}")
    print("Try manual install: pip install openai anthropic python-dotenv requests")

### Step 2: Connect to AI Model

Provider configuration is in `setup_utils.py` so every notebook can reuse the same helpers. Run the cell below to load the shared utilities.

**Note:** By default we use the GitHub Copilot proxy running on `http://localhost:7711`. The default provider is `claude` with model `claude-sonnet-4`.

**💡 Tip:** You can change the default provider and model by adding `MODULE3_PROVIDER` and `MODULE3_OPENAI_MODEL` (or `MODULE3_CLAUDE_MODEL`) to your `.env` file. See the "Optional: Switch providers" section below for details.

If you need direct OpenAI or CircuIT access, configure it with the helper functions after this cell.

In [ ]:
# Quick setup check - imports setup_utils
try:
    import importlib
    import setup_utils
    importlib.reload(setup_utils)
    from setup_utils import *
    print(f"Setup loaded. Using {AVAILABLE_PROVIDERS} with {get_default_model()}")
    print("Ready to continue.")
except ImportError:
    print("Setup not found.")
    print("Please run module-01-fundamentals.ipynb first to set up your environment.")

#### Optional: Switch providers or configure direct APIs

**Option 1: Configure via `.env` file (Recommended for persistent settings)**

Add these lines to your `.env` file in the project root to set your default provider and model:

```bash
# Set default provider (options: openai, claude, circuit)
MODULE3_PROVIDER=openai

# Set default models for each provider
MODULE3_OPENAI_MODEL=gpt-4o          # Options: gpt-5, gpt-4o, gpt-4o-mini
MODULE3_CLAUDE_MODEL=claude-sonnet-4 # Options: claude-sonnet-4, claude-4
MODULE3_CIRCUIT_MODEL=gpt-4.1        # Options: gpt-4.1, gpt-4o-mini
```

After updating `.env`, restart your notebook kernel and reload `setup_utils` to apply changes.

**Option 2: Switch for this session only**

- To switch for this session, run `set_provider("openai")`, `set_provider("claude")`, or `set_provider("circuit")`.
- To use direct OpenAI access, call `configure_openai_from_env()` after saving your API key to `.env`.
- To use CircuIT (Cisco Azure OpenAI), call `configure_circuit_from_env()` and then `set_provider("circuit")`.

In [ ]:
# Example usage (uncomment the line you need)
# set_provider("openai")
# configure_openai_from_env()
# configure_circuit_from_env(); set_provider("circuit")

### Step 3: Test Connection

Test that everything is working before we begin.

<div style="padding:12px; background:#fef3c7; border-radius:6px; border-left:4px solid #f59e0b; color:#78350f;">
<strong>💡 Tip:</strong><br><br>If you see long AI responses and the output shows "Output is truncated. View as a scrollable element" - <strong>click that link</strong> to see the full response in a scrollable view!
</div>

In [ ]:
success, response = test_connection()

if success:
    print('Module 3 setup verified. Ready to continue.')
else:
    print('Connection responded unexpectedly. If you changed providers, double-check credentials above.')


<div style="margin-top:16px; color:#15803d; padding:12px; background:#dcfce7; border-radius:6px; border-left:4px solid #22c55e;">
<style>
code {
  font-family: Consolas,"courier new";
  color:rgb(238, 13, 13);
  background-color: #f1f1f1;
  padding: 2px;
  font-size: 110%;
}
</style>
<strong>💡 What's Available?</strong> <br><br>
The <code>setup_utils.py</code> module provides these functions: <br><br>

<strong>Core Functions:</strong>
<ul>
<li><code>get_chat_completion(messages)</code> - Send prompts to AI</li>
<li><code>get_default_model()</code> - Get current model name</li>
<li><code>test_connection()</code> - Test AI connection</li>
</ul>

<strong>Provider Management:</strong>
<ul>
<li><code>AVAILABLE_PROVIDERS</code> - Tuple of available provider names</li>
<li><code>get_provider()</code> - Get current active provider</li>
<li><code>set_provider(name)</code> - Switch between providers</li>
<li><code>configure_openai_from_env()</code> - Configure direct OpenAI API</li>
<li><code>configure_circuit_from_env()</code> - Configure CircuIT (Cisco Azure)</li>
</ul>

<strong>Activity Testing Functions:</strong>
<ul>
<li><code>test_activity(file, code, variables)</code> - Test any activity template</li>
<li><code>test_activity_3_2(code, variables)</code> - Quick test for Activity 3.2</li>
<li><code>test_activity_3_3(code, variables)</code> - Quick test for Activity 3.3</li>
<li><code>test_activity_3_4(variables)</code> - Quick test for Activity 3.4 (Advanced)</li>
<li><code>get_refactor_judge_scenario()</code> - Get test data for Activity 3.4</li>
<li><code>list_activities()</code> - Show available activities</li>
</ul>

<strong>Evaluation & Progress Tracking:</strong>
<ul>
<li><code>evaluate_prompt(messages, activity_name, expected_tactics, ...)</code> - Get automated feedback on templates</li>
<li><code>extract_template_from_activity(file)</code> - Extract template from activity file</li>
<li><code>view_progress(activity_name)</code> - View evaluation history and improvement</li>
</ul>

These will be used throughout Sections 3.2, 3.3, and 3.4!
</div>


## Setup Complete!

<div style="background:#dcfce7; border-left:4px solid #22c55e; padding:20px; border-radius:8px; margin:24px 0; color:#000000;">
<style>
code {
  font-family: Consolas,"courier new";
  color:rgb(238, 13, 13);
  background-color: #f1f1f1;
  padding: 2px;
  font-size: 110%;
}
</style>
<strong style="color:#166534;">🎉 You're ready to go!</strong><br><br>
You've installed dependencies, loaded shared helpers, and verified connectivity.
<br><br>
✅ <strong style="color:#166534;">AI connection</strong> responding as expected<br>
✅ <strong style="color:#166534;">Shared utilities</strong> imported (<code>setup_utils.py</code>)<br>
✅ <strong style="color:#166534;">Activity helpers</strong> ready for hands-on practice<br>
✅ <strong style="color:#166534;">Testing functions</strong> available for template validation<br>
</div>

---

<div style="margin:24px 0; padding:20px 24px; background:linear-gradient(135deg, #f8fafc 0%, #e2e8f0 100%); border-radius:12px; border-left:5px solid #3b82f6; box-shadow:0 2px 8px rgba(0,0,0,0.1);">
  <div style="color:#1e293b; font-size:0.85em; font-weight:600; text-transform:uppercase; letter-spacing:1px; margin-bottom:8px;">⏭️ Next Section</div>
  <div style="color:#0f172a; font-size:1.15em; font-weight:700; margin-bottom:6px;">Section 3.2: Code Review Automation</div>
  <div style="color:#475569; font-size:0.95em; line-height:1.5; margin-bottom:12px;">Learn how to combine tactics strategically to build production-ready code review templates.</div>
  <a href="./3.2-code-review-automation.ipynb" style="display:inline-block; padding:8px 16px; background:#3b82f6; color:#fff; text-decoration:none; border-radius:6px; font-weight:600; font-size:0.9em; transition:all 0.2s;">Continue to Section 3.2 →</a>
</div>


> **💡 Tip:** The setup utilities are already loaded - just run `from setup_utils import *` to continue.